## ビンゴブロックエリアとルールを元にした情報

In [25]:
INF = 10000000

yel = 'yellow'
gre = 'green'
blu = 'blue'
red = 'red'
bla = 'black'

blocks_color = [yel, yel, gre, gre,  blu, blu,  red, red, bla]

block_bingo_area = [
    (0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6),
    (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
    (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6),
    (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6),
    (4, 0), (4, 1), (4, 2), (4, 3), (4, 4), (4, 5), (4, 6),
    (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 5), (5, 6),
    (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 6)]

crosscircle_position = [
    (0, 0), (0, 2), (0, 4), (0, 6),
    (2, 0), (2, 2), (2, 4), (2, 6),
    (4, 0), (4, 2), (4, 4), (4, 6),
    (6, 0), (6, 2), (6, 4), (6, 6)
]

blockcircle_position = [
    (1,1), (1,3), (1,5), (3,1), (3,5), (5,1), (5,3), (5,5)
]

first_set_block_position = [
    (0,0), (2,2), (4,0), (6,2), (0,4), (2,6), (4,4), (6,6)
]

first_set_colorblock_number = 4
first_set_blackblock_number = 6 



## 仮想ビングブロックエリアの初期化

In [28]:
# ダイクストラ法 Left

import networkx as nx # 経路探索用
import random #組み合わせよう
random.seed(42)

Graph = nx.DiGraph()

# ブロックビンゴエリアの必要な情報を仮想ビンゴブロックエリアに追加していく
Graph.add_nodes_from(block_bingo_area)

for blockcircle_num, position in enumerate(blockcircle_position, start=1):
    Graph.nodes[position]['set_block'] = False
    Graph.nodes[position]['blockcircle_number'] = blockcircle_num
    if position in {(1,1), (3,5)}:
        Graph.nodes[position]['set_blockcircle_color'] = yel
    elif position in {(1,3), (5,1)}:
         Graph.nodes[position]['set_blockcircle_color'] = gre
    elif position in {(1,5), (5,3)}:
         Graph.nodes[position]['set_blockcircle_color'] = red
    elif position in {(3,1), (5,5)}:
         Graph.nodes[position]['set_blockcircle_color'] = blu
    
    if blockcircle_num == first_set_colorblock_number:
        Graph.nodes[position]['set_block'] = True
        Graph.nodes[position]['set_blockcolor'] =  Graph.nodes[position]['set_blockcircle_color']
        
    if blockcircle_num == first_set_blackblock_number:
        Graph.nodes[position]['set_block'] = True
        Graph.nodes[position]['set_blockcolor'] = bla

colorblock = Graph.nodes[blockcircle_position[first_set_colorblock_number-1]]['set_blockcircle_color']
crosscircle_blockcolors = random.sample(blocks_color, 8)

for color, position in enumerate(first_set_block_position, start=0):
    Graph.nodes[position]['set_block'] = True
    Graph.nodes[position]['set_blockcolor'] = crosscircle_blockcolors[color]

In [29]:
count = 0
for g in Graph.nodes(data=True):
    print(g)
#     try:
#         if g[1]['set_block']:
#             count += 1
#     except:
#         pass
# count

((0, 0), {'set_block': True, 'set_blockcolor': 'yellow'})
((0, 1), {})
((0, 2), {})
((0, 3), {})
((0, 4), {'set_block': True, 'set_blockcolor': 'black'})
((0, 5), {})
((0, 6), {})
((1, 0), {})
((1, 1), {'set_block': False, 'blockcircle_number': 1, 'set_blockcircle_color': 'yellow'})
((1, 2), {})
((1, 3), {'set_block': False, 'blockcircle_number': 2, 'set_blockcircle_color': 'green'})
((1, 4), {})
((1, 5), {'set_block': False, 'blockcircle_number': 3, 'set_blockcircle_color': 'red'})
((1, 6), {})
((2, 0), {})
((2, 1), {})
((2, 2), {'set_block': True, 'set_blockcolor': 'yellow'})
((2, 3), {})
((2, 4), {})
((2, 5), {})
((2, 6), {'set_block': True, 'set_blockcolor': 'blue'})
((3, 0), {})
((3, 1), {'set_block': True, 'blockcircle_number': 4, 'set_blockcircle_color': 'blue', 'set_blockcolor': 'blue'})
((3, 2), {})
((3, 3), {})
((3, 4), {})
((3, 5), {'set_block': False, 'blockcircle_number': 5, 'set_blockcircle_color': 'yellow'})
((3, 6), {})
((4, 0), {'set_block': True, 'set_blockcolor': 'bl

In [18]:
import random
a = random.sample(blocks_color, 8)

In [19]:
a

['yellow', 'blue', 'green', 'red', 'red', 'black', 'green', 'yellow']

In [11]:
from pulp import *
import networkx as nx
g = nx.fast_gnp_random_graph(8, 0.26, 1).to_directed()
# source, sink = 0, 2 # 始点, 終点
# r = list(enumerate(g.edges()))
# m = LpProblem() # 数理モデル
# x = [LpVariable('x%d'%k, lowBound=0, upBound=1) for k, (i, j) in r] # 変数(路に入るかどうか)
# m += lpSum(x) # 目的関数
# for nd in g.nodes():
#     m += lpSum(x[k] for k, (i, j) in r if i == nd) == lpSum(x[k] for k, (i, j) in r if j == nd) + {source:1, sink:-1}.get(nd, 0) # 制約
# #     print(m)
# m.solve()
# print([(i, j) for k, (i, j) in r if value(x[k]) > 0.5])

In [30]:
Graph.nodes['set_block']

KeyError: 'set_block'

NoName:
MINIMIZE
1*x0 + 1*x1 + 1*x10 + 1*x11 + 1*x12 + 1*x13 + 1*x2 + 1*x3 + 1*x4 + 1*x5 + 1*x6 + 1*x7 + 1*x8 + 1*x9 + 0
SUBJECT TO
_C1: x0 - x1 = 1

_C2: - x0 + x1 - x11 - x13 + x2 + x3 + x4 - x8 = 0

_C3: x5 - x9 = -1

_C4: - x10 - x12 + x6 + x7 = 0

_C5: - x2 + x8 = 0

_C6: x10 - x5 - x6 + x9 = 0

_C7: x11 + x12 - x3 - x7 = 0

_C8: x13 - x4 = 0

VARIABLES
x0 <= 1 Continuous
x1 <= 1 Continuous
x10 <= 1 Continuous
x11 <= 1 Continuous
x12 <= 1 Continuous
x13 <= 1 Continuous
x2 <= 1 Continuous
x3 <= 1 Continuous
x4 <= 1 Continuous
x5 <= 1 Continuous
x6 <= 1 Continuous
x7 <= 1 Continuous
x8 <= 1 Continuous
x9 <= 1 Continuous